In [1]:
    # %%writefile msticpyconfig.yaml
# AzureSentinel:
#   Workspaces:
#     ASIHuntOMSWorkspaceV4:
#       TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
#       WorkspaceId: 52b1ab41-869e-4138-9e40-2a4457f09bf0
#     CyberSecuritySoc:
#       TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
#       WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d
#     Default:
#       TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
#       WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d

In [2]:
# Core MSTICPy initialization for Notebooks
from msticpy.nbtools import nbinit
nbinit.init_notebook(namespace=globals());

# Load query providers (typically you'll be using just one)
qry_prov = QueryProvider("AzureSentinel")

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
qry_prov.connect(WorkspaceConfig())
tables = qry_prov.schema_tables

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
import ipywidgets as widgets
from IPython.display import display

print('\nPlease select a table to analyze.\n')
tableDropdown = widgets.Dropdown(
    options=sorted(tables),
    value='OfficeActivity',
    description='Table:',
)
display(tableDropdown)


Please select a table to analyze.



Dropdown(description='Table:', index=231, options=('AACAudit', 'AACHttpRequest', 'AADDomainServicesAccountLogo…

In [5]:
df = qry_prov.exec_query(f'{tableDropdown.value} | take 1')
timestamp_col = 'TimeGenerated'

if df.empty:
    print("Table is empty, please select another table.")
else:
    print ("\nWhat kind of features/columns would you like to see be analyzed?")
    options = sorted(list(df))
    options.remove(timestamp_col)
    selected_features = nbwidgets.SelectSubset(source_items=options)
    print('\n\nWhat time frame do you want to analyze?\n')
    timeFrame = nbwidgets.QueryTime(units='day', max_before=20, before=5, max_after=1)
    timeFrame.display()

<IPython.core.display.Javascript object>


What kind of features/columns would you like to see be analyzed?




What time frame do you want to analyze?



In [21]:
df['ExtraProperties'][0] == None

True

In [14]:
from time_series_utils import check_kwargs, ts_anomalies_stl

start = (timeFrame.start)
end = (timeFrame.end)
features = selected_features.selected_values

featureDict = {}
anomalyDict = {}

if start == end:
    print("\nPlease make sure the start and end date are distinct\n")
elif len(features) == 0:
    print("\nPlease choose at least one feature\n")
else:
    for feature in features:
        raw_times_series_data = qry_prov.MultiDataSource.get_timeseries_data(
            start=start,
            end=end,
            table=f"{tableDropdown.value}",
            timestampcolumn=timestamp_col,
            aggregatecolumn=f"{feature}",
            aggregatefunction=f"dcount({feature})",
            add_query_items=f'|mv-expand {timestamp_col} to typeof(datetime), {feature} to typeof(long)',
        )
        df_time_series = raw_times_series_data[[timestamp_col, f'{feature}']]
        df_time_series = df_time_series.set_index(timestamp_col)
        anomalies = ts_anomalies_stl(df_time_series)
        featureDict[f'{feature}'] = anomalies
        anomalous_timestamps = list(anomalies[anomalies['anomalies']==1][timestamp_col])
        for timestamp in anomalous_timestamps:
            if timestamp not in anomalyDict:
                anomalyDict[timestamp] = [f'{feature}']
            else:
                anomalyDict[timestamp].append(f'{feature}')
        print(f'Timeseries for {feature} generated')
    print('Timestamp for Anomalous Features\n')
    anomalyDf = pd.DataFrame(anomalyDict.items(), columns=[timestamp_col, 'Anomalous Features'])
    anomalyDf.set_index(timestamp_col)
    display(anomalyDf)

<IPython.core.display.Javascript object>

Timeseries for AADGroupId generated


<IPython.core.display.Javascript object>

Timeseries for AADTarget generated


<IPython.core.display.Javascript object>

...

Timeseries for ExtendedProperties generated


<IPython.core.display.Javascript object>

Timeseries for ExternalAccess generated


<IPython.core.display.Javascript object>

msticpy.common.exceptions.MsticpyDataQueryError('Query:\n OfficeActivity  | project TimeGenerated,ExtraProperties,Type | where TimeGenerated >= datetime(2021-08-07T23:26:46.277913Z) | where TimeGenerated <= datetime(2021-08-13T23:26:46.277913Z) | make-series ExtraProperties=dcount(ExtraProperties) on TimeGenerated from datetime(2021-08-07T23:26:46.277913Z) to datetime(2021-08-13T23:26:46.277913Z) step 1h by Type |mv-expand TimeGenerated to typeof(datetime), ExtraProperties\n;')

In [10]:
try:
    timestamp_dropdown = widgets.Dropdown(
        options=sorted(list(anomalyDf[timestamp_col])),
        description='TimeStamp:',
        disabled=False,
    )
    print('\nSelect a timestamp to visualize anomalous features ')
    display(timestamp_dropdown)
except NameError:
    print("Anomaly Dataframe not instantiated yet.")


Select a timestamp to visualize anomalous features 


Dropdown(description='TimeStamp:', options=(Timestamp('2021-08-10 06:26:46.277913+0000', tz='UTC'), Timestamp(…

In [8]:
from msticpy.nbtools.timeseries import display_timeseries_anomolies

try:
    anomalous_features = tuple(anomalyDf.loc[anomalyDf[timestamp_col] == timestamp_dropdown.value]['Anomalous Features'])[0]
    for feature in anomalous_features:
        print(f"\nTime Series for {feature}\n")
        display_timeseries_anomolies(data=featureDict[feature], y=feature)
except NameError:
    print("Anomaly Dataframe not instantiated yet.")

Anomaly Dataframe not instantiated yet.
